In [1]:
import pandas as pd

df = pd.read_csv('disney_plus_titles.csv')
df = df.drop(columns = ['show_id', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description'])
df

,type,title
0,Movie,Duck the Halls: A Mickey Mouse Christmas Special
1,Movie,Ernest Saves Christmas
2,Movie,Ice Age: A Mammoth Christmas
3,Movie,The Queen Family Singalong
4,TV Show,The Beatles: Get Back
...,...,...
1445,Movie,X-Men Origins: Wolverine
1446,Movie,Night at the Museum: Battle of the Smithsonian
1447,Movie,Eddie the Eagle
1448,Movie,Bend It Like Beckham


In [2]:
category = pd.get_dummies(df.type)
df = pd.concat([df, category], axis=1)
df = df.drop(columns='type')
df

,title,Movie,TV Show
0,Duck the Halls: A Mickey Mouse Christmas Special,1,0
1,Ernest Saves Christmas,1,0
2,Ice Age: A Mammoth Christmas,1,0
3,The Queen Family Singalong,1,0
4,The Beatles: Get Back,0,1
...,...,...,...
1445,X-Men Origins: Wolverine,1,0
1446,Night at the Museum: Battle of the Smithsonian,1,0
1447,Eddie the Eagle,1,0
1448,Bend It Like Beckham,1,0


In [3]:
x = df['title']
y = df[['Movie', 'TV Show']].values

In [4]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.2)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = 8000, oov_token = 'x')
tokenizer.fit_on_texts(xtrain)
tokenizer.fit_on_texts(xtest)

sequen_latih = tokenizer.texts_to_sequences(xtrain)
sequen_test = tokenizer.texts_to_sequences(xtest)

maxlen = max([len(x) for x in sequen_latih])

pad_latih = pad_sequences(sequen_latih, maxlen=maxlen)
pad_test = pad_sequences(sequen_test,maxlen=maxlen )

In [6]:
import tensorflow as tf

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(input_dim=8000, input_length=maxlen, output_dim=32),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.LSTM(32, return_sequences=True, dropout = 0.25),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.LSTM(32, dropout = 0.1),
  tf.keras.layers.Dense(32),
  tf.keras.layers.Dense(2, activation='softmax')
  ])

In [7]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.76):
            print("\nAkurasi telah mencapai >75%!")
            self.model.stop_training = True
callbacks = myCallback()

In [9]:
num_epochs = 50
history = model.fit(pad_latih, 
                     ytrain,
                     epochs=num_epochs, 
#                      batch_size=4,
                     validation_data=(pad_test, ytest), 
                     callbacks=[callbacks]
                   )

Epoch 1/50
37/37 [==============================] - 6s 40ms/step - loss: 0.6287 - accuracy: 0.7190 - val_loss: 0.5898 - val_accuracy: 0.7241
Epoch 2/50
37/37 [==============================] - 1s 15ms/step - loss: 0.5896 - accuracy: 0.7259 - val_loss: 0.5860 - val_accuracy: 0.7241
Epoch 3/50
37/37 [==============================] - 1s 37ms/step - loss: 0.5823 - accuracy: 0.7259 - val_loss: 0.5832 - val_accuracy: 0.7241
Epoch 4/50
37/37 [==============================] - 1s 15ms/step - loss: 0.5716 - accuracy: 0.7259 - val_loss: 0.5677 - val_accuracy: 0.7241
Epoch 5/50
37/37 [==============================] - 1s 14ms/step - loss: 0.5215 - accuracy: 0.7293 - val_loss: 0.5506 - val_accuracy: 0.7310
Epoch 6/50
37/37 [==============================] - 1s 14ms/step - loss: 0.4279 - accuracy: 0.8043 - val_loss: 0.5185 - val_accuracy: 0.7931

Akurasi telah mencapai >75%!


In [ ]:
Nama : Muhammad Dhafi Alfaridzi
Email : M315X0844@dicoding.org
SIB_Group = M07